In [48]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [49]:
import numpy as np
import pandas as pd

In [50]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [51]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import Column, Integer, String, Float, Date, Text
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base() 

In [52]:
database_path = "hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
# engine = create_engine(f"sqlite:///HomeWork/Advanced-Data-Storage-and-Retrieval/hawaii.sqlite")
conn = engine.connect()
Base.metadata.create_all(conn)
session = Session(bind=engine)

Now, we will reflect that database (set up the metadata of what tables we have).

In [53]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [54]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurements', 'stations']

Inspect Tables
We note that we have two tables, measurement and station. We will save classes of these to be able to reference the tables. Presumably, these correspond to actual weather measurements and information about specific stations.

In [55]:
# Save references to each table
Measurement = Base.classes.measurements
Station = Base.classes.stations

In [56]:
#inspect measurement schema
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('Measurements')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station VARCHAR(255)
date TEXT
prcp FLOAT
tobs FLOAT


In [57]:
# Inspect stations schema
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('Stations')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station VARCHAR(255)
name VARCHAR(255)
latitude FLOAT
longitude FLOAT
elevation FLOAT
location VARCHAR(255)


In [58]:
# Create our session (link) from Python to the DB
session = Session(engine)

Trip Dates¶
We need to pick trip dates. We will use a January 2019 trip time, assuming that crowds will dip after the holiday season. So, we will go on January 4, 2019 and return on January 18, 2019.

We will save these as date objects.

In [59]:
#save start date and end date
start_date = dt.date(2019, 6, 4)
end_date = dt.date(2019, 6, 18)

Exploratory Analysis
We will do some basic exploratory analysis of the data first.

Precipitation Analysis
First, we will look at precipitation data for the last year.

As we don't know how dates are formatted, we will inspect them first.

In [60]:
#inspect how the dates are formatted
sample_dates = session.query(Measurement.date).limit(5).all()
for result in sample_dates:
    print(sample_dates)

In [61]:
#dates are formatted as YYYY-MM-DD
#we will get the maximum date after formatting the dates appropriately
#first is used as there are multiple measurements on the same day
max_date = session.query(func.max(func.strftime("%Y-%m-%d", Measurement.date))).limit(5).all()
max_date[0][0]

In [67]:

#access all precipitation data from the last year (as the end of the data is our endpoint, we only need one comparison)
precip_data = session.query(func.strftime("%Y-%m-%d", Measurement.date), Measurement.prcp).\
    filter(func.strftime("%Y-%m-%d", Measurement.date) >= dt.date(2016, 8, 23)).all()

In [63]:
measurements_data = pd.read_sql("SELECT * FROM Measurements", conn)
measurements_data.head()

,id,station,date,prcp,tobs


In [68]:
#load as a dataframe
precip_df = pd.DataFrame(precip_data, columns = ['date', 'precipitation'])

#set index
precip_df.set_index('date', inplace = True)

#look at dataframe
precip_df.head()

,precipitation
date,


In [64]:
# attacks = session.query(Measurement) 
# for attack in attacks:
#     print(Measurement.station)

engine.execute('SELECT * FROM stations LIMIT 5').fetchall()

[]

# Exploratory Climate Analysis

In [ ]:
#Design a query to retrieve the last 12 months of precipitation data and plot the results
#query for dates and find latest
latest_date = (session.query(Measurement.date)
                     .order_by(Measurement.date.desc())
                     .first())
print(latest_date)


latest_prcp = (session.query(Measurement.prcp)
                     .order_by(Measurement.prcp.desc())
                     .first())
print(latest_prcp)

#extract string from query object
latest_date = list(np.ravel(latest_date))[0]

#convert date string to datetime object
latest_date = dt.datetime.strptime(str(latest_date, '%Y-%m-%d'))

#extract year, month, and day as integers
latestYear = int(dt.datetime.strftime(latest_date, '%Y'))
latestMonth = int(dt.datetime.strftime(latest_date, '%m'))
latestDay = int(dt.datetime.strftime(latest_date, '%d'))

#calculate one year before latest date
yearBefore = dt.date(latestYear, latestMonth, latestDay) - dt.timedelta(days=365)

#query for dates and precipitation for the latest year
rainData = (session.query(Measurement.date, Measurement.prcp)
                  .filter(Measurement.date > yearBefore)
                  .order_by(Measurement.date)
                  .all())

#convert query object to data frame and set index
rainTable = pd.DataFrame(rainData)
rainTable = rainTable.set_index('date')

#sort by date
rainTable = rainTable.sort_index(ascending=True)

#plot with pandas
rainTable.plot(title="Precipitation: 8/23/16 - 8/23/17")

#Select only the date and prcp values.

# Calculate the date 1 year ago from the last data point in the database


# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


![precipitation](Images/precipitation.png)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

![describe](Images/describe.png)

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
